In [ ]:
# %pip install openai python-dotenv pandas

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import pandas as pd
from os import getenv

In [2]:
load_dotenv()

True

In [3]:
# client = OpenAI(
#   base_url="https://openrouter.ai/api/v1",
#   api_key=getenv("OPENROUTER_API_KEY"),
# )

client = OpenAI()

In [4]:
def generate_mcqs(context, assessment_los, num_questions=30):
    """
    Generate MCQs from a given context using an LLM.
    """
    prompt = f"""
    You are a master computer science educator that is proficient in teaching students and 
    creatting multiple-choice questions (MCQs) assessment that assesses their 
    understanding of given topics and achieves the learning outcomes of the assessment.
    Generate {num_questions} MCQs based on the following context which includes a list of topics covered in a Python programming course, 
    and the learning outcomes of the assessment.
    Each question should have 4 options, with one correct answer and three distractors.
    Please ensure the questions are relevant, clear, and cover key aspects of the context.
    The questions should be in a variety of difficulty levels (i.e. easy, medium, hard).
    
    Context: {context}
    Assessment learning outcomes: {assessment_los}
    
    Format each question strictly as follows:
    Q1. [Question text]
    A) [Option 1]
    B) [Option 2]
    C) [Option 3]
    D) [Option 4]
    Correct Answer: [Correct Option]
    Difficulty: [Difficulty Level]
    
    
    Your answer striclty must be Q1, Q2, Q3, and so on. Nothing extra, don't write any markdown in your response.
   
    """
    
    response = client.chat.completions.create(
        model="gpt-4o-2024-11-20",
        messages=[
            # {"role": "system", "content": "You are a helpful assistant that generates multiple-choice questions."},
            {"role": "user", "content": prompt}
        ]
    )
    
    mcqs = response.choices[0].message.content
    return mcqs

In [5]:
# Read the context from a text file 
context = ""
with open("content.txt", "r") as f:
    context = f.read()
    
# print(context)

In [6]:
# Read the learning outcomes from a text file 
assessment_los = ""
with open("A1_los.txt", "r") as f:
    assessment_los = f.read()
    
# print(assessment_los)

In [7]:
mcqs = generate_mcqs(context, assessment_los)

In [8]:
mcqs

'Q1. What is the first step in the six-step problem-solving process?  \nA) Analyse the problem  \nB) Identify the problem  \nC) Implement the solution  \nD) Generate potential solutions  \nCorrect Answer: B  \nDifficulty: Easy  \n\nQ2. Which of the following best defines wicked problems?  \nA) Problems that have simple solutions.  \nB) Problems that occur regularly and are easy to solve.  \nC) Problems difficult to define, with no clear solutions.  \nD) Problems that always have one correct answer.  \nCorrect Answer: C  \nDifficulty: Medium  \n\nQ3. Which programming construct is used to repeatedly execute a block of code while a condition is true?  \nA) for loop  \nB) while loop  \nC) if statement  \nD) elif statement  \nCorrect Answer: B  \nDifficulty: Easy  \n\nQ4. What does the Python function `type()` do?  \nA) Converts a variable into a different type.  \nB) Checks the size of the variable.  \nC) Returns the data type of a variable.  \nD) Assigns a type to the variable.  \nCorrec

In [9]:
mcqs = mcqs.split("\n\n")
# mcqs.pop(0)
mcqs

['Q1. What is the first step in the six-step problem-solving process?  \nA) Analyse the problem  \nB) Identify the problem  \nC) Implement the solution  \nD) Generate potential solutions  \nCorrect Answer: B  \nDifficulty: Easy  ',
 'Q2. Which of the following best defines wicked problems?  \nA) Problems that have simple solutions.  \nB) Problems that occur regularly and are easy to solve.  \nC) Problems difficult to define, with no clear solutions.  \nD) Problems that always have one correct answer.  \nCorrect Answer: C  \nDifficulty: Medium  ',
 'Q3. Which programming construct is used to repeatedly execute a block of code while a condition is true?  \nA) for loop  \nB) while loop  \nC) if statement  \nD) elif statement  \nCorrect Answer: B  \nDifficulty: Easy  ',
 'Q4. What does the Python function `type()` do?  \nA) Converts a variable into a different type.  \nB) Checks the size of the variable.  \nC) Returns the data type of a variable.  \nD) Assigns a type to the variable.  \nCo

In [16]:
# Save MCQs to a text file
with open("generated_mcqs.txt", "w") as f:
    for mcq in mcqs:
        f.write(f"{mcq}\n\n")

In [17]:

# Read MCQs back from the text file
with open("generated_mcqs.txt", "r") as f:
    mcqs_read = f.read().split("\n\n")
    # Remove empty strings
    mcqs_read = [mcq for mcq in mcqs_read if mcq]

In [19]:
# mcqs_read

In [14]:
def parse_mcqs(mcqs_list):
    parsed_data = []
    for mcq in mcqs_list:
        lines = mcq.strip().split('\n')
        if len(lines) < 6:  # Basic validation
            continue
            
        question = lines[0].strip()
        options = [lines[i].strip() for i in range(1, 5)]
        
        # Extract correct answer and difficulty
        correct_answer = ""
        difficulty = ""
        for i in range(5, len(lines)):
            if "Correct Answer:" in lines[i]:
                correct_answer = lines[i].split("Correct Answer:")[1].strip()
            elif "Difficulty:" in lines[i]:
                difficulty = lines[i].split("Difficulty:")[1].strip()
        
        parsed_data.append({
            'Question': question,
            'Option A': options[0],
            'Option B': options[1],
            'Option C': options[2],
            'Option D': options[3],
            'Correct Answer': correct_answer,
            'Difficulty': difficulty
        })
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(parsed_data)
    df.to_csv('parsed_mcqs.csv', index=False)
    return df

parsed_mcqs = parse_mcqs(mcqs)
parsed_mcqs.head()


,Question,Option A,Option B,Option C,Option D,Correct Answer,Difficulty
0,Q1. What is the first step in the six-step pro...,A) Analyse the problem,B) Identify the problem,C) Implement the solution,D) Generate potential solutions,B,Easy
1,Q2. Which of the following best defines wicked...,A) Problems that have simple solutions.,B) Problems that occur regularly and are easy ...,"C) Problems difficult to define, with no clear...",D) Problems that always have one correct answer.,C,Medium
2,Q3. Which programming construct is used to rep...,A) for loop,B) while loop,C) if statement,D) elif statement,B,Easy
3,Q4. What does the Python function `type()` do?,A) Converts a variable into a different type.,B) Checks the size of the variable.,C) Returns the data type of a variable.,D) Assigns a type to the variable.,C,Easy
4,Q5. Which operator has the highest precedence ...,A) Addition `+`,B) Parentheses `()`,C) Multiplication `*`,D) Division `/`,B,Easy


In [20]:
# print(mcqs[3])